# Fine Tuning of ResNet50

We want to fine-tune ResNet50 to classify pictorial genres.

We then want to save the model.

## Libraries

In [1]:
!pip install tensorflow-addons

In [2]:
import tensorflow_addons as tfa

In [3]:
from tensorflow.keras.models import Model
from keras.applications.resnet import ResNet50 
from keras.applications.resnet import preprocess_input as preprocess_input_resnet
from tensorflow.keras.preprocessing import image as image_resnet


import tensorflow as tf
from keras.layers import *
import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
import numpy as np
import cv2 as cv

import matplotlib.pyplot as plt
from PIL import Image
import os

from IPython.display import clear_output

## Setup

In [5]:
def google_or_kaggle(string):
    string = string.lower()
    if string == 'google':
        # Google path
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        root_dir = '/content/drive/MyDrive'
        base_dir = root_dir + '/Painting/data/'
        data_folder = base_dir + 'raw/dataset/'
    elif string == 'kaggle':
        # Kaggle path
        data_folder = '/kaggle/input/painting/'
    else:
        data_folder = ''

    return data_folder

In [20]:
VM_NAME = 'kaggle'
data_folder = google_or_kaggle(VM_NAME)

## Dataset

We have to resize to (224,224) and to preprocess all the images to be readable from ResNet50.

In [7]:
def preprocess_cv2_image_resnet(image):
  #image = cv.imread(image_path)
  image = cv.resize(image, (224, 224))
  image =  cv.cvtColor(image, cv.COLOR_BGR2RGB)
  image = Image.fromarray(image)
  image = image_resnet.img_to_array(image)
  image = np.expand_dims(image, axis = 0)
  return preprocess_input_resnet(image)

In [8]:
def preprocess_resnet(image):
  image = cv.resize(image, (224, 224))
  image =  cv.cvtColor(image, cv.COLOR_BGR2RGB)
  image = Image.fromarray( (image * 255).astype(np.uint8) )
  image = image_resnet.img_to_array(image)
  #image = np.expand_dims(image, axis = 0)
  image =  preprocess_input_resnet(image)

  image = image * 255
  image = image.astype(np.uint8)
  return image

In [9]:
train_dir = os.path.join(data_folder, "refactored_train")
test_dir = os.path.join(data_folder, "refactored_test")

In [10]:
train_datagen = ImageDataGenerator(validation_split=0.1, 
                                   preprocessing_function=preprocess_resnet)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_resnet)

In [11]:
batch_size = 32

num_classes = 42 #n_genre
input_shape = (224, 224) #(224, 224, 3)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=input_shape,
        color_mode='rgb',
        batch_size=batch_size,
        subset='training',
        class_mode='sparse',
        )

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=input_shape,
        color_mode='rgb',
        batch_size=batch_size,
        subset='validation',
        class_mode='sparse',
        )

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape,
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='sparse'
        )


## Models

In [12]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)) )
#base_model.summary()

In [13]:
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(256, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense( num_classes , activation="softmax")(head_model)

In [14]:
model = Model(inputs=base_model.input, outputs=head_model)
#model.summary()

In [15]:
for layer in base_model.layers:
	layer.trainable = False

In [16]:
f1_micro = tfa.metrics.F1Score(num_classes=num_classes, average='micro') # Unbalanced
f1_macro = tfa.metrics.F1Score(num_classes=num_classes, average='macro')

In [17]:
# compile the model
model.compile(loss="sparse_categorical_crossentropy", #sparse_categorical_crossentropy # categorical_crossentropy
              optimizer="adam",
              metrics=["accuracy", f1_micro] )

# Train

In [18]:
# Train it on the data for some epochs
epochs = 10

history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [31]:
if VM_NAME == 'kaggle':
    model.save('/kaggle/working/resnet_finetuing_model')
else:
    model.save(data_folder + 'resnet_finetuing_model')

In [32]:
if VM_NAME == 'kaggle':
    !zip -r file.zip '/kaggle/working/resnet_finetuing_model'

In [24]:
from matplotlib import pyplot as plt
#x_plot = list(range(1,epochs+1))

def plot_history(network_history):
    epochs = len( history.history['loss'] )
    x_plot = list(range(1,epochs+1))
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['accuracy'])
    plt.plot(x_plot, network_history.history['val_accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

In [25]:
plot_history(history)

In [26]:
eval_result = model.evaluate(test_generator)
print("[test loss, test accuracy]:", eval_result)

In [27]:
test_pred = model.predict(test_generator)

In [28]:
y_pred = []
for pred in test_pred:
  y_pred.append( np.argmax(pred) )

In [29]:
from sklearn.metrics import f1_score

y_true = test_generator.labels

f1_test = f1_score(y_true, y_pred, average='macro')
print('Average f1_score: {} \n' .format(f1_test) )

print('F1-SCORE FOR EACH CLASS')
print('-----------------------')
av_f1_score = f1_score(y_true, y_pred, average=None)
for i in range(len(av_f1_score)):
  print('{} : {} '.format( i, av_f1_score[i]))

In [30]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay( confusion_matrix=cm )

disp.plot()
frame1 = plt.gca()
frame1.axes.get_xaxis().set_visible(False)
plt.show()